<a href="https://colab.research.google.com/github/AnjaliRavenly/dw_matrix_road_sign/blob/master/road_sign_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin
%reload_ext tensorboard

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      vorbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      ) 
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v6(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024, activation='relu'),
  Dropout(0.3),
  Dense(1024, activation='relu'),
  Dropout(0.3),
  Dense(num_classes, activation='softmax'),
])



In [14]:
model = get_cnn_v6(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 8s 28ms/step - loss: 2.3381 - accuracy: 0.3377 - val_loss: 0.5390 - val_accuracy: 0.8273
Epoch 2/5
272/272 [==============================] - 7s 27ms/step - loss: 0.4847 - accuracy: 0.8504 - val_loss: 0.1814 - val_accuracy: 0.9445
Epoch 3/5
272/272 [==============================] - 7s 27ms/step - loss: 0.2278 - accuracy: 0.9321 - val_loss: 0.0670 - val_accuracy: 0.9790
Epoch 4/5
272/272 [==============================] - 7s 26ms/step - loss: 0.1431 - accuracy: 0.9573 - val_loss: 0.0577 - val_accuracy: 0.9828
Epoch 5/5
272/272 [==============================] - 7s 26ms/step - loss: 0.1164 - accuracy: 0.9659 - val_loss: 0.0256 - val_accuracy: 0.9929


0.9603174603174603

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),
  Dense(1024, activation='relu'),
  Dropout(0.3),
  Dense(1024, activation='relu'),
  Dropout(0.3),
  Dense(num_classes, activation='softmax'),
])

In [0]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 6ms/step - loss: 0.2262 - accuracy: 0.9485


0.9485260844230652

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  model.fit(
      X_train, 
      y_train,
      batch_size = 128, #int(params.get('batch_size', 128)),
      epochs = 5,
      verbose =0
      ) 
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

In [21]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.42900298782529284, 'dropout_cnn_block_three': 0.455324103990598, 'dropout_cnn_block_two': 0.46859692175334056, 'dropout_dense_block_one': 0.47307405724090384, 'dropout_dense_block_two': 0.33606086605951796}
accuracy=0.9614512324333191
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4250439389507681, 'dropout_cnn_block_three': 0.4624576810778951, 'dropout_cnn_block_two': 0.4653509966038555, 'dropout_dense_block_one': 0.5141640174406932, 'dropout_dense_block_two': 0.383642842963836}
accuracy=0.9621315002441406
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.4842895876528366, 'dropout_cnn_block_three': 0.34828109033997107, 'dropout_cnn_block_two': 0.43788689274867637, 'dropout_dense_block_one': 0.5883041483368991, 'dropout_dense_block_two': 0.4361695907992951}
accuracy=0.9696145057678223
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.36833993247406915, 'dropout_cnn_block_three': 0.42630279617343125, 'dropout_cnn_block_two': 0.3965291300